## Setting up

### Loading required modules

In [1]:
using NetworkInference, Assortativity, LightGraphs

### Defining paths and filenames

In [2]:
datadir = "data/"
data_filename = "data.csv"
groups_filename = "groups.tsv"

"groups.tsv"

## Use `NetworkInference.jl` to infer networks and write networks to file

### 1. Inferring networks

#### One step

In [3]:
pidc_net = infer_network(datadir * data_filename, PIDCNetworkInference(), delim = ',')
signed_pearson_net = infer_network(datadir * data_filename, CorrelationNetworkInference("Pearson", true, nothing), delim = ',')

typeof(pidc_net)

Getting nodes...
Inferring network...
Getting nodes...
Inferring network...


InferredNetwork

#### Multiple steps

In [4]:
# first get the nodes and expression values from the data
nodes, expression_values = get_nodes(datadir * data_filename, delim = ',', get_values = true)

# then infer networks
mi_net = InferredNetwork(MINetworkInference(), nodes)
unsigned_spearman_net = InferredNetwork(CorrelationNetworkInference("Spearman", false, expression_values), nodes)

typeof(unsigned_spearman_net)

InferredNetwork

### 2. Accessing network properties

In [5]:
# number of nodes and edges of an InferredNetwork
number_of_nodes = length(mi_net.nodes)
number_of_edges = length(mi_net.edges)

number_of_nodes, number_of_edges

(69, 2346)

In [6]:
# access nodes and edges of an InferredNetwork
unsigned_spearman_net.nodes[1], unsigned_spearman_net.edges[1]

(Node("ACTB", [4, 4, 4, 4, 5, 4, 4, 4, 5, 4  …  4, 4, 5, 2, 4, 4, 3, 5, 4, 3], 6, [0.021653543307086614, 0.07874015748031496, 0.17716535433070865, 0.5885826771653543, 0.1141732283464567, 0.01968503937007874]), NetworkInference.Edge(Node[Node("FGF4", [4, 4, 4, 4, 4, 4, 4, 4, 4, 4  …  1, 1, 1, 1, 1, 1, 1, 1, 1, 1], 5, [0.46062992125984253, 0.011811023622047244, 0.15748031496062992, 0.3484251968503937, 0.021653543307086614]), Node("POU5F1", [5, 4, 5, 5, 7, 5, 4, 5, 5, 5  …  1, 1, 1, 1, 1, 1, 1, 1, 1, 1], 7, [0.33661417322834647, 0.007874015748031496, 0.07086614173228346, 0.25984251968503935, 0.28346456692913385, 0.02952755905511811, 0.011811023622047244])], 0.8358907981753785))

### 3. Writing networks to file

In [7]:
# export the network to file where each line stores an edge in the form: node1 - node2 - weight of the edge
filename = "pidc_network.txt"
write_network_file(datadir * filename, pidc_net)

In [8]:
# one can also read a previously inferred network that has been exported
read_network_file(datadir * filename).nodes[1:10] # print the first 10 nodes of the imported network

10-element Array{Node,1}:
 Node("SALL4", Int64[], 0, Float64[])
 Node("LIN28", Int64[], 0, Float64[])
 Node("GLI2", Int64[], 0, Float64[]) 
 Node("VIM", Int64[], 0, Float64[])  
 Node("CDK2", Int64[], 0, Float64[]) 
 Node("CLDN6", Int64[], 0, Float64[])
 Node("PAX6", Int64[], 0, Float64[]) 
 Node("DNMT1", Int64[], 0, Float64[])
 Node("CD34", Int64[], 0, Float64[]) 
 Node("ACTB", Int64[], 0, Float64[]) 

## Using `Assortativity.jl`

### 1. Loading a network from file

In [9]:
# load the PIDC network inferred previously
pidc_network = load_network(datadir * filename)

# check the number of nodes
length(pidc_network.nodes)

69

### 2. Converting an `InferredNetwork` to a `SimpleGraph`

This produces a SimpleGraph storing the edges from the InferredNetwork in no particular order. Each node is represented by a number, which is why the `ids_to_genes` object is used to keep track of nodes.

In [10]:
# get groups annotations i.e. which node labels belongs to which group
genes_to_groups = get_labels_to_groups(pidc_network.nodes, datadir * groups_filename)

# assign an index to each group
groups_to_indices = get_groups_to_indices(genes_to_groups)

thread = 1 warning: only found 11 / 12 columns on data row: 19. Filling remaining columns with `missing`


Dict{Symbol,Int64} with 12 entries:
  :Cell_Cycle          => 1
  :Core_Pluripotency   => 3
  :Primed_Pluripotency => 9
  :Chromatin_Modulator => 2
  :Endoderm            => 4
  :Trophoectoderm      => 12
  :Loading_Control     => 5
  :Neuroectoderm       => 8
  :Mesoderm            => 6
  :Naive_Pluripotency  => 7
  :Signalling          => 11
  :Primitive_Endoderm  => 10

In [11]:
# convert an InferredNetwork to a SimpleGraph
pidc_graph, ids_to_genes = InferredNetwork_to_LightGraph(pidc_network)

({69, 2346} undirected simple Int64 graph, Dict(68 => "LIFR",2 => "LIN28",11 => "BMPR1A",39 => "BMP4",46 => "DPPA4",25 => "REST",55 => "MYST3",42 => "HES1",29 => "PTPN11",58 => "TCF3"…))

One can then access the SimpleGraph properties e.g.

In [12]:
number_of_edges = ne(pidc_graph)
number_of_nodes = nv(pidc_graph)

number_of_edges, number_of_nodes

(2346, 69)

The correspondence between the SimpleGraph and the genes works this way:

In [13]:
first_edge = collect(edges(pidc_graph))[1] # look at the first edge in the network
source = first_edge.src # source of the edge
destination = first_edge.dst # destination of the edge

println("The first edge in the graph is $(ids_to_genes[source]) => $(ids_to_genes[destination]).")

The first edge in the graph is SALL4 => LIN28.


### 3. Calculating the assortativity coefficient

#### Assortativity coefficient of a `SimpleGraph` type

First load the network at a given threshold number of edges; this is done on the InferredNetwork because the SimpleGraph does not keep track of edges order.

In [14]:
# load the PIDC network at a threshold of 150 edges
pidc_network_150 = set_threshold(pidc_network, 150)

# then convert the network at that threshold to a SimpleGraph
pidc_graph_150, ids_to_genes = InferredNetwork_to_LightGraph(pidc_network_150)

({67, 150} undirected simple Int64 graph, Dict(2 => "FGF5",11 => "DNMT3B",39 => "CDK2",46 => "SOX2",25 => "NR0B1",55 => "TBX3",42 => "SETDB1",66 => "PRMT6",58 => "UTF1",29 => "TRP53"…))

Then calculate the assortativity coefficient of the graph. Given only a `SimpleGraph`, the `assortativity` and `second_neighbour_assortativity` functions will return the degree assortativity, and one needs to give the dictionaries as illustrated below as arguments in order for them to return *label* assortativity calculated from group *labels* (here being referred to as **functional assortativity** because groups were chosen to describe biological *functions*).

In [15]:
# calculate the degree assortativity of the graph
degree_assortativity = assortativity(pidc_graph_150)
excess_degree_assortativity = assortativity(pidc_graph_150, excess_degree = true)
second_neighbour_degree_assortativity = second_neighbour_assortativity(pidc_graph_150)
second_neighbour_excess_degree_assortativity = second_neighbour_assortativity(pidc_graph_150, excess_degree = true)

# calculate the functional assortativity of the graph
functional_assortativity = assortativity(pidc_graph_150, genes_to_groups, groups_to_indices, ids_to_genes)
second_neighbour_functional_assortativity = second_neighbour_assortativity(pidc_graph_150, genes_to_groups, groups_to_indices, ids_to_genes)

┌ Warning: 7 edges were omitted out of 150 considered (at least one node with excess degree of 0)
└ @ Assortativity C:\Users\leo-d\.julia\packages\Assortativity\1tn7S\src\measures.jl:61
┌ Warning: 24 edges were omitted out of 1472 considered (at least one node with excess degree of 0)
└ @ Assortativity C:\Users\leo-d\.julia\packages\Assortativity\1tn7S\src\measures.jl:164


AssortativityObject(0.14659736650356758, [8 52 … 6 4; 52 248 … 28 14; … ; 6 28 … 52 0; 4 14 … 0 0], Dict(:Cell_Cycle => 1,:Primed_Pluripotency => 9,:Core_Pluripotency => 3,:Naive_Pluripotency => 7,:Chromatin_Modulator => 2,:Trophoectoderm => 12,:Neuroectoderm => 8,:Loading_Control => 5,:Signalling => 11,:Endoderm => 4…))

The excess degree assortativity may return a warning, as illustrated above, in case the excess degree for a given node is 0 (this means the degree of that node is 1) and as a result the connectivity matrix cannot be updated.

These functions return an AssortativityObject which is used to store specific information about the assortativty coefficient. Its properties can be accessed as follows:

In [16]:
@show functional_assortativity.value # value of the assortativity coefficient
@show functional_assortativity.connectivity # connectivity matrix
@show functional_assortativity.groups # groups present in the connectivity matrix

typeof(functional_assortativity)

functional_assortativity.value = 0.1834038054968288
functional_assortativity.connectivity = [0 8 4 0 1 0 0 1 0 0 0 0; 8 26 12 0 4 0 0 8 2 0 7 1; 4 12 10 0 2 0 14 8 1 0 5 2; 0 0 0 0 0 0 0 0 1 0 2 0; 1 4 2 0 0 0 0 0 0 0 3 0; 0 0 0 0 0 0 0 0 0 0 0 0; 0 0 14 0 0 0 30 1 0 0 7 0; 1 8 8 0 0 0 1 18 1 0 6 0; 0 2 1 1 0 0 0 1 6 0 2 0; 0 0 0 0 0 0 0 0 0 0 0 0; 0 7 5 2 3 0 7 6 2 0 4 0; 0 1 2 0 0 0 0 0 0 0 0 0]
functional_assortativity.groups = Dict(:Cell_Cycle => 1,:Primed_Pluripotency => 9,:Core_Pluripotency => 3,:Naive_Pluripotency => 7,:Chromatin_Modulator => 2,:Trophoectoderm => 12,:Neuroectoderm => 8,:Loading_Control => 5,:Signalling => 11,:Endoderm => 4)


AssortativityObject

Not all groups from the original dataset may be present in the graph as is the case here (this is because only the top 150 edges from the original network are included and these do not connect nodes from all groups). The `AssortativityObject` can be filtered out for groups not currently present for clarity by doing:

In [17]:
filter_connectivity(functional_assortativity)

AssortativityObject(0.1834038054968288, [0 8 … 0 0; 8 26 … 7 1; … ; 0 7 … 4 0; 0 1 … 0 0], Dict(:Cell_Cycle => 1,:Core_Pluripotency => 3,:Primed_Pluripotency => 8,:Naive_Pluripotency => 6,:Chromatin_Modulator => 2,:Loading_Control => 5,:Neuroectoderm => 7,:Trophoectoderm => 10,:Signalling => 9,:Endoderm => 4…))

#### Assortativity coefficient of a `NetworkInference` type

This package also supports the dispatch of the `assortativity` and `second_neighbour_assortativity` functions directly on the `NetworkInference` type as illustrated below. The main difference with manually calculating the assortativity coefficient on a `SimpleGraph` is that the following functions do not return the `ids_to_genes` (at the moment).

In [18]:
### degree assortativity

# dispatch direction on a NetworkInference type
@show assortativity(pidc_network_150).value 

# one can set a threshold on the fly
@show assortativity(pidc_network, 150)

# or a range of thresholds
assortativities = assortativity(pidc_network, (100:50:200))
@show [assortativity.value for assortativity in assortativities]

# the syntax is similar for the second neighbour flavour of the function
@show second_neighbour_assortativity(pidc_network_150).value
@show second_neighbour_assortativity(pidc_network, 150).value
@show second_neighbour_assortativity(pidc_network, 150, excess_degree = true)
assortativities = second_neighbour_assortativity(pidc_network, (100:50:200))
[assortativity.value for assortativity in assortativities]

(assortativity(pidc_network_150)).value = -0.020952380952380945
assortativity(pidc_network, 150) = AssortativityObject(-0.020952380952380945, [0 1 6 0 0 0 0 0 0 0 0; 1 0 4 1 7 2 2 1 0 0 0; 6 4 6 2 8 9 5 3 1 0 1; 0 1 2 0 7 2 1 3 3 0 1; 0 7 8 7 8 8 4 8 3 1 1; 0 2 9 2 8 0 2 4 2 1 0; 0 2 5 1 4 2 2 4 1 0 0; 0 1 3 3 8 4 4 14 10 5 4; 0 0 1 3 3 2 1 10 2 2 3; 0 0 0 0 1 1 0 5 2 0 1; 0 0 1 1 1 0 0 4 3 1 0], nothing)
[assortativity.value for assortativity = assortativities] = [0.08432338452622433, -0.020952380952380945, -0.010523965050386627]
(second_neighbour_assortativity(pidc_network_150)).value = -0.008506008973770207
(second_neighbour_assortativity(pidc_network, 150)).value = -0.008506008973770207
second_neighbour_assortativity(pidc_network, 150, excess_degree=true) = AssortativityObject(-0.010231071193595532, [8 38 14 32 20 10 10 4 0 0; 38 48 28 64 30 18 60 24 10 8; 14 28 12 30 18 10 56 20 10 8; 32 64 30 72 40 52 104 46 14 18; 20 30 18 40 8 20 56 30 14 8; 10 18 10 52 20 20 30 8 2 8; 10 60 56

┌ Warning: 24 edges were omitted out of 1472 considered (at least one node with excess degree of 0)
└ @ Assortativity C:\Users\leo-d\.julia\packages\Assortativity\1tn7S\src\measures.jl:164


3-element Array{Float64,1}:
  0.05750754689299194 
 -0.008506008973770207
 -0.008424999514993757

In [19]:
### functional assortativity

# works on the same way as above
@show assortativity(pidc_network_150, genes_to_groups, groups_to_indices).value
@show assortativity(pidc_network, genes_to_groups, groups_to_indices, 150).value
assortativities = assortativity(pidc_network, genes_to_groups, groups_to_indices, (100:50:200))
[assortativity.value for assortativity in assortativities]

@show second_neighbour_assortativity(pidc_network_150, genes_to_groups, groups_to_indices).value
@show second_neighbour_assortativity(pidc_network, genes_to_groups, groups_to_indices, 150).value
assortativities = second_neighbour_assortativity(pidc_network, genes_to_groups, groups_to_indices, (100:50:200))
[assortativity.value for assortativity in assortativities]

(assortativity(pidc_network_150, genes_to_groups, groups_to_indices)).value = 0.1834038054968288
(assortativity(pidc_network, genes_to_groups, groups_to_indices, 150)).value = 0.1834038054968288
(second_neighbour_assortativity(pidc_network_150, genes_to_groups, groups_to_indices)).value = 0.14659736650356758
(second_neighbour_assortativity(pidc_network, genes_to_groups, groups_to_indices, 150)).value = 0.14659736650356758


3-element Array{Float64,1}:
 0.1839972911836493 
 0.14659736650356758
 0.10564879262303448

### 4. Calculating other measures with `LightGraphs.jl`

One can look at other properties of the graph to put the assortativity coefficient in context, e.g.

In [20]:
clustering_coefficient = global_clustering_coefficient(pidc_graph_150)
communities_number = get_communities_number(pidc_graph_150)
graph_modularity = get_modularity(pidc_graph_150)
degree_sequence = degree(pidc_graph_150)
centrality = betweenness_centrality(pidc_graph_150)

clustering_coefficient, communities_number, graph_modularity, degree_sequence, centrality

(0.3953804347826087, 8, 0.5808, [6, 5, 4, 8, 5, 3, 11, 2, 8, 8  …  9, 1, 6, 5, 3, 3, 2, 1, 2, 1], [0.12268266047677814, 0.055696329813976875, 0.056643356643356645, 0.22378601485964708, 0.05086083344721765, 0.04285719559475216, 0.06955152583906182, 0.05994793333028629, 0.06226149773866762, 0.0667933494054432  …  0.04903812064651226, 0.0, 0.015363585635078848, 0.11955317733289758, 0.034215470758045846, 0.04123398509864984, 0.01229215396636211, 0.0, 0.030303030303030304, 0.0])

### 5. Noise and randomness

The `Assortativity.jl` package also implements a few functions to add noise in different objects, as illustrated below.

In [21]:
# rewire 50 edges (edges are swapped two by two, 25 times) at random
rewired_graph, rewired_ids_to_genes, rewired_edges = random_edge_rewiring(pidc_network_150, 25)

({67, 150} undirected simple Int64 graph, Dict(2 => "FGF5",11 => "DNMT3B",39 => "CDK2",46 => "SOX2",25 => "NR0B1",55 => "TBX3",42 => "SETDB1",66 => "PRMT6",58 => "UTF1",29 => "TRP53"…), 50)

In [22]:
# randomly delete 10 nodes from the network
random_node_deletion_graph, random_node_deletion_ids_to_genes = random_node_deletion(pidc_network_150, 10)

({57, 121} undirected simple Int64 graph, Dict(2 => "CDH2",11 => "ACTB",39 => "RCC1",46 => "MYC",25 => "MKI67IP",55 => "GRB2",42 => "SOX2",29 => "REST",8 => "POU5F1",57 => "NR6A1"…))

In [23]:
# make a random graph with as many nodes and edges as the one given in input
random_graph, random_ids_to_genes = random_network(pidc_network_150)

({67, 150} undirected simple Int64 graph, Dict(2 => "FGF5",11 => "DNMT3B",39 => "CDK2",46 => "SOX2",25 => "NR0B1",55 => "TBX3",42 => "SETDB1",66 => "PRMT6",58 => "UTF1",29 => "TRP53"…))

In [24]:
# change the value of 20 genes (i.e. change their groups) in the genes_to_groups dictionary
randomised_genes_to_groups, randomised_groups_count = randomise_annotations(genes_to_groups, 20)

(Dict("SALL4" => :Core_Pluripotency,"LIN28" => :Core_Pluripotency,"GLI2" => :Neuroectoderm,"NCAM1" => :Neuroectoderm,"VIM" => :Neuroectoderm,"CDK2" => :Cell_Cycle,"CLDN6" => :Primed_Pluripotency,"PAX6" => :Neuroectoderm,"DNMT1" => :Core_Pluripotency,"CD34" => :Primed_Pluripotency…), 20)

One can then calculate the assortativity coefficient of these randomised objects:

In [25]:
@show functional_assortativity.value # original functional assortativity coefficient

rewired_functional_assortativity = assortativity(rewired_graph, genes_to_groups, groups_to_indices, rewired_ids_to_genes)
random_node_deletion_functional_assortativity = assortativity(random_node_deletion_graph, genes_to_groups, groups_to_indices, random_node_deletion_ids_to_genes)
random_functional_assortativity = assortativity(random_graph, genes_to_groups, groups_to_indices, random_ids_to_genes)
randomised_groups_functional_assortativity = assortativity(pidc_graph_150, randomised_genes_to_groups, groups_to_indices, ids_to_genes)

(rewired_functional_assortativity.value, random_node_deletion_functional_assortativity.value,
random_functional_assortativity.value, randomised_groups_functional_assortativity.value)

functional_assortativity.value = 0.1834038054968288

(0.16754756871035942, 0.2193871221206732, 0.006439742410303623, 0.07701934609167228)

### 6. Printing and writing a network to JSON

In [26]:
JSON_network = InferredNetwork_to_JSON(pidc_network_150, genes_to_groups, groups_to_indices)

Dict{String,Array{T,1} where T} with 2 entries:
  "nodes" => JSON_node[JSON_node("CLDN6", :Primed_Pluripotency, 9), JSON_node("…
  "edges" => JSON_edge[JSON_edge("CLDN6", "IGF2", 1.99259), JSON_edge("FGF5", "…

JSON formatted networks can easily be printed to the REPL by running `using JSON; JSON.print(stdout, JSON_network, 4)`.

One can then export them:

In [27]:
write_JSON_network(JSON_network, "$(datadir)pidc_network_150.json")